In [3]:
from pyspark.sql import SparkSession
from datetime import datetime

spark = SparkSession \
        .builder \
        .getOrCreate()
spark

25/02/10 13:30:12 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServ

# W5 M2
hdfs에 파일을 업로드 합니다.

In [4]:
!pwd
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-03.parquet
!hdfs dfs -mkdir /yellow_texi
!hdfs dfs -put ./yellow_tripdata_2024-03.parquet /yellow_texi/2024-03.parquet
!rm ./yellow_tripdata_2024-03.parquet

/usr
--2025-02-05 12:24:28--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-03.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 108.138.245.225, 108.138.245.96, 108.138.245.16, ...
connected. to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|108.138.245.225|:443... 
HTTP request sent, awaiting response... 200 OK
Length: 60078280 (57M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-03.parquet’

yellow_tripdata_202 100%[===================>]  57.29M  12.4MB/s    in 5.2s    

2025-02-05 12:24:34 (11.1 MB/s) - ‘yellow_tripdata_2024-03.parquet’ saved [60078280/60078280]

Picked up _JAVA_OPTIONS: -Xms512m -Xmx8g
2025-02-05 12:24:35,185 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
mkdir: `/yellow_texi': File exists
Picked up _JAVA_OPTIONS: -Xms512m -Xmx8g
2025-02-05 12:24:36,158 WARN util.NativeCodeLoader: Unable to load native-ha

In [4]:
df = spark.read.parquet("hdfs://master:9000/yellow_texi/2024-03.parquet")
df.columns
df.dtypes

[('VendorID', 'int'),
 ('tpep_pickup_datetime', 'timestamp_ntz'),
 ('tpep_dropoff_datetime', 'timestamp_ntz'),
 ('passenger_count', 'bigint'),
 ('trip_distance', 'double'),
 ('RatecodeID', 'bigint'),
 ('store_and_fwd_flag', 'string'),
 ('PULocationID', 'int'),
 ('DOLocationID', 'int'),
 ('payment_type', 'bigint'),
 ('fare_amount', 'double'),
 ('extra', 'double'),
 ('mta_tax', 'double'),
 ('tip_amount', 'double'),
 ('tolls_amount', 'double'),
 ('improvement_surcharge', 'double'),
 ('total_amount', 'double'),
 ('congestion_surcharge', 'double'),
 ('Airport_fee', 'double')]

## Data Cleaning
다음으로는 결측치가 있는 행을 제거합니다.

In [5]:
df = df.dropna(how='any')

In [6]:
df.head(30)

[Row(VendorID=1, tpep_pickup_datetime=datetime.datetime(2024, 3, 1, 0, 18, 51), tpep_dropoff_datetime=datetime.datetime(2024, 3, 1, 0, 23, 45), passenger_count=0, trip_distance=1.3, RatecodeID=1, store_and_fwd_flag='N', PULocationID=142, DOLocationID=239, payment_type=1, fare_amount=8.6, extra=3.5, mta_tax=0.5, tip_amount=2.7, tolls_amount=0.0, improvement_surcharge=1.0, total_amount=16.3, congestion_surcharge=2.5, Airport_fee=0.0),
 Row(VendorID=1, tpep_pickup_datetime=datetime.datetime(2024, 3, 1, 0, 26), tpep_dropoff_datetime=datetime.datetime(2024, 3, 1, 0, 29, 6), passenger_count=0, trip_distance=1.1, RatecodeID=1, store_and_fwd_flag='N', PULocationID=238, DOLocationID=24, payment_type=1, fare_amount=7.2, extra=3.5, mta_tax=0.5, tip_amount=3.0, tolls_amount=0.0, improvement_surcharge=1.0, total_amount=15.2, congestion_surcharge=2.5, Airport_fee=0.0),
 Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2024, 3, 1, 0, 9, 22), tpep_dropoff_datetime=datetime.datetime(2024, 3, 1, 0

In [7]:
from pyspark.sql import functions as F

# 날짜 컬럼 추가
df_with_date = df.withColumn("date", F.to_date("tpep_dropoff_datetime"))

# 일별 집계 계산
daily_stats = df_with_date.groupBy("date").agg(
    F.count("date").alias("trip_count"),
    F.sum("total_amount").alias("total_revenue"),
    F.sum("trip_distance").alias("total_trip_distance")
).orderBy("date")

# 전체 통계 계산
total_stats = df_with_date.agg(
    F.count("date").alias("trip_count"),
    F.sum("total_amount").alias("total_revenue"),
    F.sum("trip_distance").alias("total_trip_distance")
).collect()[0]


In [8]:
trip_count = total_stats["trip_count"]
total_revenue = total_stats["total_revenue"]
total_trip_distance = total_stats["total_trip_distance"]
avg_trip_distance = total_trip_distance / trip_count

daily_stats_list = daily_stats.collect()

In [9]:
print("total number of trips:", trip_count)
print("total revenue generated from the trips:", total_revenue)
print("average trip distance:", avg_trip_distance)

print("\ntrips per day, average:", trip_count / 30)
for row in daily_stats_list:
    print(row["date"], ":", row["trip_count"])

print("\ntotal revenue per day, average:", total_revenue / 30)
for row in daily_stats_list:
    print(row["date"], ":", row["total_revenue"])

total number of trips: 3156438
total revenue generated from the trips: 87206005.4001112
average trip distance: 3.528815848750871

trips per day, average: 105214.6
2002-12-31 : 1
2003-01-01 : 1
2024-02-29 : 3
2024-03-01 : 107450
2024-03-02 : 105853
2024-03-03 : 88641
2024-03-04 : 90131
2024-03-05 : 104366
2024-03-06 : 112720
2024-03-07 : 111078
2024-03-08 : 108321
2024-03-09 : 113265
2024-03-10 : 89972
2024-03-11 : 91713
2024-03-12 : 102685
2024-03-13 : 109637
2024-03-14 : 119433
2024-03-15 : 113272
2024-03-16 : 111779
2024-03-17 : 90429
2024-03-18 : 89046
2024-03-19 : 102483
2024-03-20 : 107455
2024-03-21 : 114209
2024-03-22 : 103508
2024-03-23 : 98813
2024-03-24 : 87379
2024-03-25 : 86382
2024-03-26 : 100465
2024-03-27 : 106854
2024-03-28 : 108492
2024-03-29 : 101079
2024-03-30 : 95911
2024-03-31 : 83002
2024-04-01 : 608
2024-04-02 : 2

total revenue per day, average: 2906866.8466703733
2002-12-31 : 18.0
2003-01-01 : 25.15
2024-02-29 : 60.91
2024-03-01 : 2879916.639999919
2024-03-02 :